In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tools import BaseK, show_plot, Solver, Differ, Comparator
from matplotlib.ticker import FuncFormatter
from matplotlib.widgets import Slider, Button
import copy

In [2]:
%matplotlib qt

In [3]:
class K(BaseK):
    light = 0.0001
    l = 1e9  # 8 10
    l_ = 1e5  # 5 6
    # Qt
    qE = 1e9  # 8 10
    qE_ = 1e8  # 7 9
    H = 1e9  # 8 10

    qH = 1e6  # 2000
    redQ = 1e9
    oxQ = 1

    qQD = 2

    qPh = 1e-5

    D = 1
    D_ = 0.05

    r = 1e9
    p = 0.0001

    rD = 1e9

In [4]:
T = np.linspace(0, 0.001, 100)

In [5]:
def system_full(t, C, k=K()):
    [Q, Qt, DH, Q_DH, QH, D, QHH, QHD, QD] = C
    C1 = Q
    C2 = Qt
    C3 = DH
    C4 = Q_DH
    C5 = QH
    C6 = D
    C7 = QHH
    C8 = QHD
    C9 = QD

    R1 = k.l * k.light * C1
    R2 = k.l_ * C2
    R3 = k.qE * C2 * C3
    R4 = k.qE_ * C4
    R5 = k.H * C4
    R6 = k.qH * C2 * C7
    R7 = k.redQ * C5 * C5
    R8 = k.oxQ * C1 * C7
    R9 = k.qQD * C2 * C8
    R10 = k.qPh * C2
    R11 = k.D * C1 * C6
    R12 = k.D_ * C9
    R13 = k.r * C5 * C6
    R14 = k.p * C8
    R15 = k.rD * C6 * C6

    res = dict(
        dCl=-R1 + R2 + R7 - R8 - R11 + R12,
        dC2=R1 - R2 - R3 + R4 - R6 - R9 - R10,
        dC3=-R3 + R4,
        dC4=R3 - R4 - R5,
        dC5=R5 + 2 * R6 - 2 * R7 + 2 * R8 + R9 - R13,
        dC6=R5 - R11 + R12 - R13 - 2 * R15,
        dC7=-R6 + R7 - R8 + R14,
        dC8=-R9 + R13 - R14,
        dC9=R9 + R11 - R12,
    )

    return list(res.values())


c_full = '[Q, Qt, DH, Q_DH, QH, D, QHH, QHD, QD]'
initial_full = [1, 0, 1, 0, 0, 0, 0, 0, 0]

In [6]:
def system_ions(t, C, k=K()):
    [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp] = C
    re = k.qE * k.light * Q * DH
    rH = k.H * Qm * DHp
    rr = k.r * QH * D
    rp = k.p * QHD
    rqH = k.qH * Q * QHH
    rdisp = k.redQ * QH * QH
    rph = k.qPh * Q
    rs = k.qQD * Q * QHD
    rd = k.D_ * QD
    rc = k.D * Q * D
    rrD = k.rD * D * D

    Q = -re - rqH - rph - rs - rc + rdisp + rd
    DH = -re
    QHH = -rqH + rp + rdisp
    D = -rr - rc - 2 * rrD + rH + rd
    QHD = -rp - rs + rr
    QH = -rr - 2 * rdisp + rH + 2 * rqH + rs
    QD = -rd + rs + rc
    Qm = -rH + re
    DHp = -rH + re

    return [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]


c_ions = '[Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]'
initial_ions = [1, 1, 0, 0, 0, 0, 0, 0, 0]

In [12]:
def system_base(t, C, k=K()):
    [Q, Qt, DH, QH, D, QHH] = C
    C1 = Q
    C2 = Qt
    C3 = DH
    C5 = QH
    C6 = D
    C7 = QHH

    R1 = k.l * k.light * C1
    R2 = k.l_ * C2
    R3 = k.H * C2 * C3
    R6 = k.qH * C2 * C7
    R7 = k.redQ * C5 * C5
    R13 = k.r * C5 * C6
    R15 = k.rD * C6 * C6

    res = dict(
        dCl=-R1 + R2 + R7,
        dC2=R1 - R2 - R3 - R6,
        dC3=-R3,
        dC5=R3 + 2 * R6 - 2 * R7 - R13,
        dC6=R3 - R13 - 2 * R15,
        dC7=-R6 + R7,
    )

    return list(res.values())


c_base = '[Q, Qt, DH, QH, D, QHH]'
initial_base = [1, 0, 1, 0, 0, 0]

#   Plot

In [14]:
# Y= Solver(system_ions, K(), initial_ions, c_ions, T)
# Y = Solver(system_full, K(), initial_full, c_full, T)
# Y = Differ(
#     Solver(system_full, K(), initial_full, c_full, T),
#     Solver(system_ions, K(), initial_ions, c_ions, T),
# )
# Y = Differ(
#     Solver(system_1, K(), init0, c0, T),
#     Solver(system_2, K(), init0, c0, T),
# )
Y = Comparator(
    Solver(system_full, K(), initial_full, c_full, T),
    Solver(system_base, K(), initial_base, c_base, T)

)
LIM = None
AUTO = False

In [15]:
# Plots

fig, ax = plt.subplots()
gs = plt.GridSpec(2, 2, figure=fig)
fig.subplots_adjust(left=0.25, right=0.99, bottom=0.2, top=0.95, hspace=0.1, wspace=0.1)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Plots
scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')
K_base = Y.K
Y.solve()

plots = {}
# base_ax = plt.subplot(gs[0, 0:2])
# for D in ['A', 'B']:
#     plots[D] = base_ax.plot(Y.T, Y[D], label=D)[0]
# base_ax.legend()
# base_ax.xaxis.set_major_formatter(scal)
# if LIM is not None:
#     base_ax.set_ylim([-lims, lims])

base_ax = plt.subplot(gs[0, 0:2])
for c in ['Q', 'DH', 'QHH']:
    plots[c] = base_ax.plot(Y.T, Y[c], label=c)[0]
base_ax.legend()
base_ax.xaxis.set_major_formatter(scal)
if LIM is not None:
    base_ax.set_ylim([-LIM, LIM])

d_ax = plt.subplot(gs[1, 0])
plots['D'] = d_ax.plot(Y.T, Y['D'], label='D')[0]
d_ax.legend()
d_ax.xaxis.set_major_formatter(scal)
if LIM is not None:
    d_ax.set_ylim([-LIM, LIM])

# other_ax = plt.subplot(gs[1, 1])
# other_ax.xaxis.set_major_formatter(scal)
# for c in ['QH', 'QHD']:
#     plots[c] = other_ax.plot(Y.T, Y[c], label=c)[0]
# other_ax.legend()
# other_ax.xaxis.set_major_formatter(scal)
# if LIM is not None:
#     other_ax.set_ylim([-LIM, LIM])


# Sliders
sliders = {}
buttons = {}
K_new = copy.deepcopy(K_base)


def resolve(event=None):
    Y.solve(K=K_new)
    for c, plot in plots.items():
        plot.set_ydata(Y[c])
    fig.canvas.draw_idle()


def get_slider_action(k):
    def update(val):
        K_new[k] = K_base[k] * 10**val
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        if AUTO:
            resolve()

    return update


def get_zero_button_action(k):
    def update(event):
        K_new[k] = 0
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        fig.canvas.draw_idle()
        if AUTO:
            resolve()

    return update


def reset(event):
    global K_new
    K_new = copy.deepcopy(K_base)
    for k in sliders:
        sliders[k].reset()
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
    fig.canvas.draw_idle()
    if AUTO:
        resolve()


solve_axes = fig.add_axes([0.02, 0.05, 0.05, 0.05])
solve_button = Button(solve_axes, 'Solve', hovercolor='0.975')
solve_button.on_clicked(resolve)

reset_axes = fig.add_axes([0.08, 0.05, 0.05, 0.05])
reset_button = Button(reset_axes, 'Reset', hovercolor='0.975')
reset_button.on_clicked(reset)

for i, k in enumerate(K_base):
    k_axes = fig.add_axes(
        [
            0.03,  # left
            0.95 - 0.04 * i,  # bottom
            0.10,  # width
            0.03,  # height
        ]
    )
    amp_slider = Slider(
        ax=k_axes,
        label=k,
        valmin=-3,
        valmax=3,
        valinit=0,
        valstep=0.1,
        orientation="horizontal",
    )
    sliders[k] = amp_slider
    amp_slider.on_changed(get_slider_action(k))

    k_zero_axes = fig.add_axes(
        [
            0.15,  # left
            0.955 - 0.04 * i,  # bottom
            0.05,  # width
            0.02,  # height
        ]
    )
    zero_button = Button(k_zero_axes, f"{K_new[k]: .1e}", hovercolor='0.975')
    buttons[k] = zero_button
    zero_button.on_clicked(get_zero_button_action(k))

In [49]:
zero_button.label.set_text('1')

In [ ]:
def system_activated(t, C, k=K):
    [Qt, Qs, DH, QHH, D, QHD, QH, QD] = C
    Qs = -R1
    Qt = R1 - R2 - R3 - R4 - R5
    DH = -R2
    QH = R2 + 2 * R3 + R4 - R6 - 2 * R7
    D = R2 - R5 - R6 + R8 - 2 * R10
    QHH = -R3 + R7 + R9
    QHD = -R4 + R6 - R9
    QD = R4 + R5 - R8
    Q = R7 + R8 - R11

    re = k['Ke'] * k['light'] * Q * DH
    rr = k['Kr'] * QH * D
    rp = k['Kp'] * QHD
    rqH = k['KqH'] * Q * QHH
    rdisp = k['Kdisp'] * QH * QH
    rph = k['Kph'] * Q
    rs = k['Ks'] * Q * QHD
    rd = k['Kd'] * QD
    rc = k['Kc'] * Q * D
    rrD = k['KrD'] * D * D

    R1 = k['e'] * k['light'] * Qs
    R2 = k['Ke'] * Qt * DH
    R3 = k['KqH'] * Q * QHH
    R4 = k['Ks'] * Qt * QHD
    R5 = k['Kr'] * Qt * D
    R6 = k['Kr'] * QH * D
    R7 = k['Kdisp'] * QH * QH
    R8 = k['Kd'] * QD
    R9 = k['Kp'] * QHD
    R10 = k['KrD'] * D * D

    return [Q, DH, QHH, D, QHD, QH, QD]